In [1]:
import sys
sys.path.append("./keras/classification/")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  
from keras.preprocessing.image import ImageDataGenerator
from Models_MNV2 import model_version_mb2
from log_visualization_tool import visual_train
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (/home/zhangyouan/anaconda3/envs/Tensorflow/lib/python3.8/site-packages/tensorflow/python/eager/context.py)

In [2]:

def cls_model_build(data_path=r"/home/zhangyouan/桌面/zya/dataset/681/PCScreen_Book_PhoneScreen", 
                    epoch=300, 
                    save_model_name = 'pc_book_phone_0904_lower_train.h5',
                    model_summary="True",
                    weights = None):
    
    model = model_version_mb2()
    if weights is not None:
        model.load_weights(weights)
        
    test_data_path = data_path + "val"
    train_data_path = data_path + "train"
    validation_datagen = ImageDataGenerator(rescale=1. / 255)
    training_datagen = ImageDataGenerator(rescale=1. / 255,
                                          rotation_range=40, # 旋转范围
                                          width_shift_range=0.2, # 宽平移
                                          height_shift_range=0.2,# 高平移
                                          shear_range=0.2, # 剪切
                                          zoom_range=0.2, # 缩放
                                          horizontal_flip=True,
                                          fill_mode='nearest')
    
    training_generator = training_datagen.flow_from_directory(train_data_path,
                                                              target_size=(120, 160),
                                                              color_mode="grayscale",
                                                              class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(test_data_path,
                                                                  target_size=(120, 160),
                                                                  color_mode="grayscale",
                                                                  class_mode='categorical')
    print(validation_generator.class_indices)

    # 使用 ReduceLROnPlateau 回调函数
    adam = Adam(lr=0.01)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
    model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #======== 模型训练 =========
    history = model.fit(
        training_generator,
        epochs = epoch,
        validation_data = validation_generator,
        callbacks=[reduce_lr, early_stopping]
    )

    model.save(save_model_name) # model 保存

    return history

In [3]:
if __name__ == "__main__":
    history = cls_model_build(data_path=r"/home/zhangyouan/桌面/zya/dataset/681/PCScreen_Book_PhoneScreen/", 
                              epoch=300, 
                              save_model_name = './trained_model/pc_book_phone_0904_lower_train.h5' )
    visual_train(history)

Instructions for updating:
Colocations handled automatically by placer.


TypeError: unsupported operand type(s) for /: 'Dimension' and 'float'

In [2]:
from keras.preprocessing.image import ImageDataGenerator

data_path=r"/home/zhangyouan/桌面/zya/dataset/681/PCScreen_Book_PhoneScreen/"
test_data_path = data_path + "val"
validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = validation_datagen.flow_from_directory(test_data_path,
                                                                  target_size=(120, 160),
                                                                  color_mode="grayscale",
                                                                  class_mode='categorical')
print(validation_generator.class_indices)

Found 600 images belonging to 3 classes.
{'PcScreen': 0, 'PhoneScreen': 1, 'book': 2}


In [7]:
import os 
import tensorflow as tf
import numpy as np
print("Tensorflow version:", tf.__version__)
from PIL import Image

# 制作量化数据集
path = r"/home/zhangyouan/桌面/zya/dataset/681/PCScreen_Book_PhoneScreen/train/"
list_dir = os.listdir(path)

# labels = {"paper":0, "rock":1, "scissors":2}
labels = {'PcScreen': 0, 'PhoneScreen': 1, 'book': 2}

test_images = []
test_images_link = []
test_labels = []
for i in list_dir:
    path1 = path + i + "/"
    list_label = os.listdir(path1)
    # print(list_label)
    for j in list_label:
        path2 = path1 + j
        tmp = [0, 0, 0]
        tmp[labels[i]]=1
        test_labels.append(tmp)
        test_images_link.append(path2)
        test_images_tmp = Image.open(path2)
        test_images_g = test_images_tmp.convert('L')
        test_images_g_resize = test_images_g.resize((160, 120), Image.ANTIALIAS) # (width, height)
        test_images.append(np.array(test_images_g_resize))

test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_images = np.expand_dims(test_images, axis=-1)
test_images = test_images.astype(np.float32) # / 255.0
test_images = test_images / 255.0  # 将数值范围压缩到0~1之间；
# test_images = test_images/127.5-1

model = tf.keras.models.load_model(r'/home/zhangyouan/桌面/zya/NN_net/network/SSD/IMX_681_ssd_mobilenet_git/keras/classification/trained_model/pc_book_phone_0904.h5')
# model.summary()
scores = model.evaluate(test_images, test_labels, verbose=0)
print("模型准确度：",scores[1])

Tensorflow version: 2.13.0


/tmp/ipykernel_20086/2656379968.py:29: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  test_images_g_resize = test_images_g.resize((160, 120), Image.ANTIALIAS) # (width, height)


模型准确度： 0.9998196363449097
